## Data Cleaning

In [12]:
import pandas as pd 
import numpy as np 
from pymongo import MongoClient
import json

### Database Connection

In [13]:
DB_USERNAME = 'anutibara'
DB_PASSWORD = 'anutibara'
DB_HOST = 'scraping-cluster-7dtgt.gcp.mongodb.net'
DB_NAME = 'scraping_db'
try:
    client = MongoClient(f'mongodb+srv://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}')
    print("Database connected successfully")
except Exception as e:
    print("Error to connect to database: ", e)
db = client.get_database('scraping_db')
properties = db.properties

Database connected successfully


### Database Queries

##### Total Documents

In [48]:
properties.count_documents({})

41645

### Convert Mongo Collection to DataFrame

In [43]:
documents = pd.DataFrame(list(properties.find()))